# AI-Powered, Region-Aware Validation of Radiology Reports Using ACR Guidelines and Customer Rubrics



Radiology reporting demands consistent, standards-aligned documentation to ensure diagnostic clarity and reduce variability across clinicians. Manually checking each report against anatomy-specific ACR(American College of Radiology) guidelines and institution/customer rubrics is accurate but time-consuming and difficult to scale.

This project implements an AI-driven radiology report validator using CrewAI(https://docs.crewai.com/) orchestration with a FloTorch-backed LLM, designed as a two-stage pipeline that mirrors real-world reviewer workflow.

**Workflow:**

1. A lightweight classifier agent reads the report text and predicts a single anatomical region key (e.g., chest, abdomen).
2. The system then verifies whether both an ACR guideline document and a matching customer rubric exist for that region in Google Drive (PDF/DOCX).
3. When both standards are available, the validation agent loads them via secure tool calls (used only as internal context) and produces an adherence report with weighted scoring (ACR + rubric) and actionable improvement items.
4. If either document is missing or fails to load, the system automatically suppresses scoring and returns a “Not Applicable” evaluation that explains the limitation—avoiding unreliable judgments and preventing guideline leakage.

By combining region-aware document retrieval, strict output enforcement, and robust gateway retry handling, this workflow reduces reviewer effort, improves reporting consistency, and supports scalable quality checks for radiology documentation.

## Installation of libraries


* Installs all required libraries for AI agent orchestration (CrewAI +
FloTorch) and for parsing guideline documents in PDF and DOCX formats.

In [1]:
# Install core orchestration + PDF + DOCX support

# flotorch[crewai]:
#   - Provides the FloTorch LLM gateway integration
#   - Includes CrewAI-compatible LLM wrappers used by agents
#
# crewai:
#   - Core multi-agent orchestration framework
#   - Used to define Agents, Tasks, Crews, and execution flow
#
# crewai_tools:
#   - Utility decorators and helpers for defining tools
#   - Enables agents to safely call Python functions (e.g., loading guidelines)
#
# pdfplumber:
#   - Extracts text from PDF documents
#   - Used to read ACR guideline PDFs
#
# python-docx:
#   - Reads and parses DOCX files
#   - Used for customer rubrics or guideline documents stored as Word files
#
# -q flag:
#   - Quiet installation to reduce notebook output noise

!pip install -q flotorch[crewai] crewai crewai_tools pdfplumber python-docx


## Import libraries



Imports core libraries, document parsers, CrewAI primitives, and configures FloTorch LLM gateway settings.

In [2]:
# -----------------------------
# Standard library imports
# -----------------------------
import contextlib  # redirect stdout/stderr (silence noisy agent runs)
import io          # in-memory text buffers for capturing logs
import json        # parse JSON tool inputs safely
import os          # file and directory operations
import re          # regex for parsing/cleaning text and filenames
import time        # sleep/backoff for transient retries
from typing import Any, Dict, List, Optional, Tuple  # type hints
from getpass import getpass

# PDF + DOCX parsing
import pdfplumber            # read guideline PDFs
from docx import Document    # read rubric DOCX files


# CrewAI primitives + tool decorator
from crewai import Agent, Task, Crew, Process  # agent orchestration
from crewai.tools import tool                  # expose Python tools to agents
from flotorch.crewai.llm import FlotorchCrewAILLM

## FloTorch LLM Configuration for CrewAI

Prompts the user for FloTorch configuration values and securely captures the API key, validating that the key is provided before proceeding.

In [3]:
FLOTORCH_GATEWAY_BASE_URL = input("Enter FloTorch Gateway URL: ").strip()
FLOTORCH_MODEL_NAME = input("Enter FloTorch model name: ").strip()
FLOTORCH_API_KEY = getpass("Enter FloTorch API key: ").strip()

if not FLOTORCH_API_KEY:
    raise ValueError("FloTorch API key cannot be empty.")

Enter FloTorch Gateway URL: https://gateway.flotorch.cloud
Enter FloTorch model name: flotorch/claude-sonnet-45
Enter FloTorch API key: ··········


In [4]:
model  =  FlotorchCrewAILLM(
        model_id=FLOTORCH_MODEL_NAME,
        api_key=FLOTORCH_API_KEY,
        base_url=FLOTORCH_GATEWAY_BASE_URL,
    )

print(f"Flotorch LLM model configured: {model.model}")

Flotorch LLM model configured: flotorch/claude-sonnet-45


## Mount Google Drive for supporting documents

Mounts Google Drive, defines guideline/rubric folders, lists supported documents, and derives available anatomical region keys from filenames.
Please make sure you download the two folders "rad-guidelines-acr" and "rad-guidelines-rubric" and upload them into your Google Drive to make them available.

In [5]:
from google.colab import drive

# Mount Google Drive to access guideline and rubric files
drive.mount("/content/drive")


# -----------------------------
# Folder locations
# -----------------------------

ACR_DIR = "/content/drive/MyDrive/rad-guidelines-acr"       # modify it with your exact location
RUBRIC_DIR = "/content/drive/MyDrive/rad-guidelines-rubric"  # modify it with your exact location

# Ensure folders exist (no overwrite)
os.makedirs(ACR_DIR, exist_ok=True)
os.makedirs(RUBRIC_DIR, exist_ok=True)


def _list_docs(folder: str) -> List[str]:
    """List supported PDF/DOCX files in a folder."""
    if not os.path.isdir(folder):
        return []
    return [
        f for f in os.listdir(folder)
        if f.lower().endswith((".pdf", ".docx"))
    ]


def extract_region_from_filename(filename: str) -> str:
    """
    Extract region key from filename using first token.
    e.g., 'chest_acr_2022.pdf' -> 'chest'
    """
    base = os.path.splitext(filename)[0].lower()
    return re.split(r"[_\-\s]+", base)[0].strip()


# List available documents
ACR_FILES = _list_docs(ACR_DIR)
RUBRIC_FILES = _list_docs(RUBRIC_DIR)

# Derive unique region keys from filenames
ACR_REGIONS = sorted({extract_region_from_filename(f) for f in ACR_FILES})
RUBRIC_REGIONS = sorted({extract_region_from_filename(f) for f in RUBRIC_FILES})


# Debug print: folders, files, and detected regions
print("📁 ACR folder:", ACR_DIR)
print("   Files:", ACR_FILES)
print("   Regions:", ACR_REGIONS)

print("\n📁 Rubric folder:", RUBRIC_DIR)
print("   Files:", RUBRIC_FILES)
print("   Regions:", RUBRIC_REGIONS)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📁 ACR folder: /content/drive/MyDrive/rad-guidelines-acr
   Files: ['Chest_acr.pdf']
   Regions: ['chest']

📁 Rubric folder: /content/drive/MyDrive/rad-guidelines-rubric
   Files: ['Chest_Radiology_Report_Rubric_Template.docx']
   Regions: ['chest']


## Document Loading & Input Normalization Helpers

Utility functions that normalize region inputs, safely read PDF/DOCX files, clean extracted text, and load the correct guideline or rubric document for a given anatomical region.

In [6]:
# ----------------------------
# Helpers to stabilize tool I/O
# ----------------------------

def _normalize_region_key(x: Any) -> str:
    """
    Normalize region_key passed by CrewAI tools.
    Handles string, dict, or JSON-string inputs.
    """
    if x is None:
        return ""

    # Plain string or JSON string
    if isinstance(x, str):
        s = x.strip()
        if s.startswith("{") and s.endswith("}"):
            try:
                obj = json.loads(s)
                if isinstance(obj, dict) and "region_key" in obj:
                    return str(obj["region_key"]).strip().lower()
            except Exception:
                pass
        return s.lower()

    # Dict input
    if isinstance(x, dict) and "region_key" in x:
        return str(x["region_key"]).strip().lower()

    # Fallback
    return str(x).strip().lower()


def _clean_text(text: str) -> str:
    """Normalize whitespace and remove null characters."""
    if not text:
        return ""
    text = text.replace("\x00", " ")
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()


def _read_pdf_text(path: str, max_pages: int = 3, max_chars: int = 12000) -> str:
    """Read first N pages of a PDF with length cap."""
    parts = []
    with pdfplumber.open(path) as pdf:
        for i in range(min(max_pages, len(pdf.pages))):
            parts.append(pdf.pages[i].extract_text() or "")
    txt = _clean_text("\n\n".join(parts))
    return txt[:max_chars]


def _read_docx_text(path: str, max_chars: int = 12000) -> str:
    """Read DOCX text with length cap."""
    doc = Document(path)
    txt = _clean_text("\n".join(p.text for p in doc.paragraphs))
    return txt[:max_chars]


def _load_doc_text(folder: str, region_key: Any, files: List[str]) -> Tuple[bool, str]:
    """
    Load the first matching document for a region.
    Returns (success, text_or_error).
    """
    key = _normalize_region_key(region_key)

    if not key:
        return False, "Empty region_key."

    # Match file by extracted region key
    candidates = [f for f in files if extract_region_from_filename(f) == key]
    if not candidates:
        return False, f"No document found for region '{key}'."

    path = os.path.join(folder, candidates[0])

    try:
        if path.lower().endswith(".pdf"):
            text = _read_pdf_text(path)
        elif path.lower().endswith(".docx"):
            text = _read_docx_text(path)
        else:
            return False, f"Unsupported file type: {path}"
    except Exception as e:
        return False, f"Error reading {path}: {e}"

    if not text:
        return False, f"Could not extract text from {path}."

    return True, text

## CrewAI Tools for Standard Documents Retrieval

CrewAI tool wrappers that load ACR guidelines or customer rubrics for a given region and return the text or a clear “not found” message for controlled agent usage.

In [7]:
@tool
def load_acr_guidelines(region_key: str) -> str:
    """CrewAI tool to load ACR guideline text for a region."""
    ok, text_or_error = _load_doc_text(ACR_DIR, region_key, ACR_FILES)
    return text_or_error if ok else f"NO_ACR_FOUND: {text_or_error}"


@tool
def load_rubric(region_key: str) -> str:
    """CrewAI tool to load customer rubric text for a region."""
    ok, text_or_error = _load_doc_text(RUBRIC_DIR, region_key, RUBRIC_FILES)
    return text_or_error if ok else f"NO_RUBRIC_FOUND: {text_or_error}"

## Agent 1: Anatomical Region Classifier

A strict CrewAI classifier that analyzes the radiology report and returns exactly one valid anatomical region key, enforcing format and allowlist constraints to prevent ambiguous or invalid outputs.

In [8]:
def infer_anatomical_structure(report_text: str, llm) -> str:
    """
    Agent 1 (Classifier):
    - Input: raw radiology report text
    - Output: ONE lowercase region key (plain text), e.g. "chest"

    Strict mode:
    - No keyword safety-net extraction
    - No default fallback
    - If output is not exactly one allowed region key, raise ValueError
    """

    # Minimal agent whose only job is to output a region key
    classifier_agent = Agent(
        role="Anatomical Structure Classifier",
        goal="Identify the single primary anatomical region for a radiology report.",
        backstory=(
            "You map radiology reports to broad region keys used to select guideline docs. "
            "Return ONLY the region key."
        ),
        tools=[],            # no tools needed for classification
        llm=llm,             # shared FloTorch LLM instance
        verbose=False,       # suppress extra agent logs
    )

    # Single task: strict one-word output contract
    task = Task(
        description=(
            "You are given a radiology report below.\n\n"
            f"{report_text}\n\n"
            "Return ONLY ONE lowercase anatomical region key.\n"
            "Rules:\n"
            "- One word only\n"
            "- Lowercase\n"
            "- No punctuation\n"
            "- No explanation\n"
            "- Must be one of: chest, abdomen, pelvis, brain, spine, cardiac, renal, liver, echo\n"
        ),
        agent=classifier_agent,
        expected_output="A single lowercase word like 'chest'",
    )

    # Run a tiny crew with just this agent + task
    crew = Crew(
        # ... keep your existing crew params here (process, memory, etc.)
        tasks=[task],
        verbose=False,       # suppress crew traces
    )

    # Execute and normalize
    raw = str(crew.kickoff()).strip().lower()

    # Enforce strict "one token, allowlisted" output
    allowed = {
        "chest", "abdomen", "pelvis", "brain",
        "spine", "cardiac", "renal", "liver", "echo",
    }

    # Must be exactly one word token (no spaces/newlines)
    token = raw.split()
    if len(token) != 1:
        raise ValueError(
            f"Classifier returned invalid format (expected 1 word). Got: {raw!r}"
        )

    region = token[0]

    # Must be in allowlist
    if region not in allowed:
        raise ValueError(
            f"Classifier returned unknown region (not in allowlist). Got: {region!r}"
        )

    return region


## Anatomical Region Resolution & Normalization

Post-processes the classifier output to reliably resolve a valid anatomical region key using normalization, keyword matching, and report-text heuristics when needed.

In [9]:
def extract_anatomical_structure_from_output(raw_output: str, report_text: str) -> str:
    """
    Parse and normalize classifier output into a valid region_key.
    Falls back to report-text heuristics if needed.
    """

    # Allowed canonical region keys
    allowed = {
        "chest", "abdomen", "pelvis", "brain", "spine",
        "renal", "cardiac", "echo", "neck", "head",
        "extremity", "msk",
    }

    # Normalize raw classifier output
    raw = (raw_output or "").strip().lower()

    # Remove markdown/code fences if present
    raw = raw.replace("```", " ").strip()

    # Remove punctuation and split into tokens
    raw_clean = re.sub(r"[^a-z_\s]", " ", raw)
    tokens = [t for t in raw_clean.split() if t]

    # 1) Return first token that matches an allowed region
    for t in tokens:
        if t in allowed:
            return t

    # 2) If output is a sentence, search for allowed keywords
    for k in allowed:
        if k in raw:
            return k

    # 3) Heuristic fallback: scan report text for common synonyms
    text = (report_text or "").lower()
    synonyms = [
        ("chest", "chest"),
        ("thorax", "chest"),
        ("lungs", "chest"),
        ("abdomen", "abdomen"),
        ("abdominal", "abdomen"),
        ("pelvis", "pelvis"),
        ("brain", "brain"),
        ("head", "head"),
        ("spine", "spine"),
        ("renal", "renal"),
        ("kidney", "renal"),
        ("cardiac", "cardiac"),
        ("heart", "cardiac"),
        ("echo", "echo"),
    ]
    for needle, canonical in synonyms:
        if needle in text:
            return canonical

    # Final fallback if nothing matches
    return "chest"


## Agent 2: Standards-Based Radiology Report Validation

Creates the validator agent + task + crew that loads ACR and rubric docs (only when both exist) and outputs a strict, non-leaking validation report with weighted scoring.

In [10]:
def create_radiology_validator_crew(llm):
    # Agent responsible for validating report structure/quality against standards
    validator_agent = Agent(
        role="Radiology Report Validator",
        goal=(
            "Evaluate radiology report documentation quality using standards. "
            "Use guideline documents ONLY as internal reference."
        ),
        backstory=(
            "You are a senior radiologist assessing documentation quality only. "
            "Never quote or expose guideline/rubric text. "
            "Score only when both standards exist."
        ),
        tools=[load_acr_guidelines, load_rubric],  # tools to load ACR + rubric text
        llm=llm,                                   # shared FloTorch LLM
        verbose=False,                             # keep logs quiet
    )

    # Task prompt enforces:
    # - when to call tools
    # - when scoring is allowed
    # - strict output template (no markdown/traces)
    validate_task = Task(
        description=(
            "You are validating a radiology report.\n\n"
            "Inputs:\n"
            "- region_key: {region_key}\n"
            "- report_text: {report_text}\n"
            "- acr_available: {acr_available}\n"
            "- rubric_available: {rubric_available}\n"
            "- scoring_weights: {scoring_weights}\n"
            "- weight_acr: {weight_acr}\n"
            "- weight_rubric: {weight_rubric}\n\n"

            "STRICT RULES:\n"
            "1) Tool outputs are INTERNAL CONTEXT ONLY.\n"
            "2) NEVER paste, quote, or summarize guideline/rubric text.\n"
            "3) NEVER output URLs or citations from the guideline docs.\n"
            "4) DO NOT include 'Thought:', 'Action:', 'Observation:' lines.\n"
            "5) DO NOT use markdown or code fences (NO ```).\n"
            "6) Output MUST be ONLY the report text following the template below.\n\n"

            "DECISION LOGIC:\n"
            "A) If acr_available == true AND rubric_available == true:\n"
            "   - Call load_acr_guidelines(region_key) and load_rubric(region_key)\n"
            "   - Validate internally against BOTH\n"
            "   - Compute acr_score, rubric_score, overall_score (weighted)\n"
            "   - Output numeric scores\n\n"
            "B) Otherwise:\n"
            "   - DO NOT call tools\n"
            "   - Do NOT judge quality\n"
            "   - Overall Adherence Score must be Not Applicable\n\n"

            "OUTPUT TEMPLATE (EXACT — DO NOT ADD EXTRA SECTIONS):\n"
            "=== Radiology Validation Report ===\n"
            "Region: {region_key}\n"
            "ACR Used: <Yes/No>\n"
            "Rubric Used: <Yes/No>\n"
            "Weights: ACR=<weight_acr>, Rubric=<weight_rubric>\n"
            "Overall Adherence Score: <XX>/100 OR Not Applicable\n"
            "ACR Score: <XX>/100 OR N/A\n"
            "Rubric Score: <XX>/100 OR N/A\n"
            "\n"
            "Documentation Quality Assessment:\n"
            "- <bullet>\n"
            "- <bullet>\n"
            "- <bullet>\n"
            "\n"
            "Areas for Improvement:\n"
            "- <bullet>\n"
            "- <bullet>\n"
            "\n"
            "Notes:\n"
            "- <optional bullet>\n"
        ),
        agent=validator_agent,  # runs this task
        expected_output="Plain-text report only, exactly matching the template.",
    )

    # Sequential crew with one agent/task (easy to extend later)
    return Crew(
        agents=[validator_agent],
        tasks=[validate_task],
        process=Process.sequential,
        verbose=False,  # keep crew output quiet
    )


## Validator Output Enforcement & Consistency Guardrails

Applies deterministic checks to clean the validator output, enforce the required report template, and ensure scoring is consistent with document availability.

In [11]:
def enforce_validator_output(raw_output: str, acr_available: bool, rubric_available: bool) -> str:
    """
    Enforce stable plain-text output:
    - remove ``` fences + Thought/Action/Observation lines
    - ensure ONLY ONE Overall Adherence Score line (never both NA + numeric)
    - allow numeric scoring only if docs exist + tools ok + output matches template
    """

    txt = (raw_output or "").strip()

    # 1) Remove markdown fences
    txt = txt.replace("```", "").strip()

    # 2) Remove leaked trace lines (don't truncate whole output)
    kept = []
    for line in txt.splitlines():
        s = line.strip().lower()
        if s.startswith("thought:") or s.startswith("action:") or s.startswith("observation:"):
            continue
        kept.append(line)
    txt = "\n".join(kept).strip()

    # Detect tool-load failure hints
    tool_failed_markers = [
        "not successfully loaded",
        "could not be loaded",
        "failed to load",
        "no_acr_found",
        "no_rubric_found",
        "could not extract text",
        "error reading",
    ]
    tool_failed = any(m in txt.lower() for m in tool_failed_markers)

    # Template sanity check (prevents scoring on junk outputs)
    required_markers = [
        "=== radiology validation report ===",
        "region:",
        "acr used:",
        "rubric used:",
        "weights:",
        "documentation quality assessment:",
        "areas for improvement:",
        "notes:",
    ]
    looks_like_template = all(m in txt.lower() for m in required_markers)

    # Scoring allowed only if docs exist + tools ok + template ok
    allow_scoring = bool(acr_available and rubric_available and not tool_failed and looks_like_template)

    # Matchers for overall score lines
    score_line_num = re.compile(
        r"^\s*Overall Adherence Score:\s*(\d{1,3})\s*/\s*100\s*$",
        re.IGNORECASE,
    )
    score_line_na = re.compile(
        r"^\s*Overall Adherence Score:\s*Not Applicable\s*$",
        re.IGNORECASE,
    )

    lines = txt.splitlines()

    # Extract numeric overall score if present and valid
    numeric_score = None
    for line in lines:
        m = score_line_num.match(line)
        if m:
            val = int(m.group(1))
            if 0 <= val <= 100:
                numeric_score = val
            break

    # If scoring is allowed but numeric score missing => force Not Applicable
    if allow_scoring and numeric_score is None:
        allow_scoring = False

    # Remove ALL existing "Overall Adherence Score" lines (NA or numeric)
    body = [ln for ln in lines if not (score_line_num.match(ln) or score_line_na.match(ln))]

    # Choose exactly ONE overall line
    if allow_scoring:
        overall_line = f"Overall Adherence Score: {numeric_score}/100"
        # If model had ACR/Rubric as N/A, keep as-is (no contradiction)
    else:
        overall_line = "Overall Adherence Score: Not Applicable"

        # Remove numeric ACR/Rubric scores to avoid contradictions
        body = [ln for ln in body if not re.match(r"^\s*ACR Score:\s*\d", ln, re.IGNORECASE)]
        body = [ln for ln in body if not re.match(r"^\s*Rubric Score:\s*\d", ln, re.IGNORECASE)]

        # Force ACR/Rubric score lines to N/A if present
        fixed = []
        for ln in body:
            if ln.strip().lower().startswith("acr score:"):
                fixed.append("ACR Score: N/A")
            elif ln.strip().lower().startswith("rubric score:"):
                fixed.append("Rubric Score: N/A")
            else:
                fixed.append(ln)
        body = fixed

    # Insert overall line right after Weights (preferred)
    out = []
    inserted = False
    for ln in body:
        out.append(ln)
        if (not inserted) and ln.strip().lower().startswith("weights:"):
            out.append(overall_line)
            inserted = True

    # Fallback placement if Weights line missing
    if not inserted:
        out.insert(0, overall_line)

    return "\n".join(out).strip()


## End-to-End Radiology Validation Orchestration

Orchestrates the full validation flow—region classification, document availability checks, CrewAI validator execution, retry on off-template output, and final output enforcement.

In [12]:
def _looks_like_template(txt: str) -> bool:
    """Basic check that output matches the required template structure."""
    t = (txt or "").lower()
    required = [
        "=== radiology validation report ===",
        "region:",
        "acr used:",
        "rubric used:",
        "weights:",
        "documentation quality assessment:",
        "areas for improvement:",
        "notes:",
    ]
    return all(x in t for x in required)


def _preflight_load_docs(region_key: str):
    """
    Preflight load docs using the plain Python loader (NOT the @tool wrappers).
    Returns: (acr_available, rubric_available)
    """
    acr_ok, acr_text_or_err = _load_doc_text(ACR_DIR, region_key, ACR_FILES)
    rub_ok, rub_text_or_err = _load_doc_text(RUBRIC_DIR, region_key, RUBRIC_FILES)
    return acr_ok, rub_ok


def run_full_radiology_validation(
    report_text: str,
    scoring_weights: Dict[str, float] = {"acr": 0.5, "rubric": 0.5},
    silence_console: bool = True,
) -> str:
    # Refresh file lists (in case Drive contents changed)
    global ACR_FILES, RUBRIC_FILES
    ACR_FILES = _list_docs(ACR_DIR)
    RUBRIC_FILES = _list_docs(RUBRIC_DIR)

    # 1) Classify region (one-word contract)
    raw_classification = infer_anatomical_structure(report_text, model)
    region_key = extract_anatomical_structure_from_output(raw_classification, report_text)

    # 2) Preflight load docs (real availability, not just filenames)
    acr_available, rubric_available = _preflight_load_docs(region_key)

    # 3) Normalize weights to sum to 1.0
    acr_w = float(scoring_weights.get("acr", 0.5))
    rub_w = float(scoring_weights.get("rubric", 0.5))
    total = acr_w + rub_w
    if total <= 0:
        acr_w, rub_w, total = 0.5, 0.5, 1.0
    weights = {"acr": acr_w / total, "rubric": rub_w / total}

    # 4) Build validator crew
    validator_crew = create_radiology_validator_crew(model)

    inputs = {
        "region_key": region_key,
        "report_text": report_text,
        "acr_available": acr_available,
        "rubric_available": rubric_available,
        "scoring_weights": weights,
        "weight_acr": weights["acr"],
        "weight_rubric": weights["rubric"],
    }

    def _kickoff():
        if silence_console:
            buf = io.StringIO()
            with contextlib.redirect_stdout(buf), contextlib.redirect_stderr(buf):
                return validator_crew.kickoff(inputs=inputs)
        return validator_crew.kickoff(inputs=inputs)

    # 5) Run + retry once if off-template
    result = _kickoff()
    raw = str(result)

    if not _looks_like_template(raw):
        result = _kickoff()
        raw = str(result)

    # 6) Enforce final output contract
    return enforce_validator_output(
        raw_output=raw,
        acr_available=acr_available,
        rubric_available=rubric_available,
    )


## Test Cases

Test case 1: Defines a sample chest radiology report, sets scoring weights, runs the full validation pipeline end-to-end, and prints the final structured validation report output.

In [13]:
# ---------------------------------------------------------------
# Test Case: Chest Radiograph (ACR + Rubric available)
#
# Expected behavior:
# - Agent 1 classifies region as "chest"
# - System finds BOTH:
#     ✔ ACR chest guideline
#     ✔ Customer chest rubric
# - Validator agent:
#     ✔ Calls both tools (ACR + rubric)
#     ✔ Compares report against both
#     ✔ Returns detailed feedback
#     ✔ Returns an adherence score
# ---------------------------------------------------------------

# Sample chest radiology report
test_query = (
    "FINAL REPORT. "
    "EXAMINATION: CHEST (PORTABLE AP). "
    "INDICATION: Evaluation of endotracheal tube placement. "
    "TECHNIQUE: Portable AP radiograph of the chest. "
    "COMPARISON: None. "
    "FINDINGS: Endotracheal tube tip terminates approximately 4.5 cm above the carina. "
    "Enteric tube courses below the diaphragm with tip projecting over the stomach. "
    "Cardiomediastinal silhouette is within normal limits for portable technique. "
    "Lungs are clear without focal consolidation, pleural effusion, or pneumothorax. "
    "No acute osseous abnormality identified."
)

print("=== Running Chest Validation Test (ACR + Rubric present) ===\n")

# Optional: customize scoring weights (user-controlled)
scoring_weights = {
    "acr": 0.6,      # 60% weight to ACR compliance
    "rubric": 0.4,   # 40% weight to customer rubric
}

# Run the full validation pipeline
result = run_full_radiology_validation(
    report_text=test_query,
    scoring_weights=scoring_weights,
)

print(result)


=== Running Chest Validation Test (ACR + Rubric present) ===

[EventBus Error] Handler 'on_task_started' failed for event 'TaskStartedEvent': 'NoneType' object has no attribute 'key'
=== Radiology Validation Report ===
Region: chest
ACR Used: Yes
Rubric Used: Yes
Weights: ACR=0.6, Rubric=0.4
Overall Adherence Score: 87/100
ACR Score: 95/100
Rubric Score: 74/100

Documentation Quality Assessment:
- Appropriate imaging study ordered for specific clinical indication (endotracheal tube placement evaluation), aligning with ACR principles that exclude routine imaging
- Comprehensive anatomical coverage with all required chest structures systematically evaluated
- Quantitative measurement provided for endotracheal tube position (4.5 cm above carina)
- Clear documentation of negative findings including absence of consolidation, pleural effusion, and pneumothorax
- Appropriate acknowledgment of portable technique limitations on cardiac silhouette assessment

Areas for Improvement:
- Add a separ

 Test case 2: Defines a sample abdominal ultrasound (gallbladder) report, sets scoring weights, runs the full validation pipeline end-to-end, and prints the final structured validation report output.

In [14]:
# ---------------------------------------------------------------
# Test Case: Abdominal Ultrasound – Gallbladder (ACR + Rubric if available)
#
# Expected behavior:
# - Agent 1 classifies region as "abdomen" (or "gallbladder" → abdomen)
# - System checks:
#     ✔ ACR abdominal ultrasound guideline (if present)
#     ✔ Customer abdominal / gallbladder rubric (if present)
# - Validator agent:
#     ✔ Calls available tools (ACR + rubric only if BOTH exist)
#     ✔ Compares report against available standards
#     ✔ Returns detailed feedback
#     ✔ Returns an adherence score ONLY if both ACR + rubric exist
# ---------------------------------------------------------------

# Sample abdominal ultrasound (gallbladder) radiology report
test_query = (
    "FINDINGS: THE GALLBLADDER IS WELL VISUALIZED AND NO INTRALUMINAL STONE, "
    "WALL THICKENING OR PERICHOLECYSTIC FLUID SEEN. COMMON HEPATIC DUCT IS NORMAL "
    "IN CALIBER AT 3MM. AORTA, IVC, AND IMAGED SEGMENTS OF THE PANCREAS ARE NORMAL. "
    "THE LIVER AND SPLEEN ARE NORMAL. THE RIGHT KIDNEY MEASURES 12.7 CM IN LENGTH. "
    "THE LEFT KIDNEY MEASURES 12.6 CM. THERE IS NO HYDRONEPHROSIS OR RENAL MASS."
)

print("=== Running Abdominal Ultrasound (Gallbladder) Validation Test ===\n")

# Optional: customize scoring weights (user-controlled)
scoring_weights = {
    "acr": 0.6,      # 60% weight to ACR compliance
    "rubric": 0.4,   # 40% weight to customer rubric
}

# Run the full validation pipeline
result = run_full_radiology_validation(
    report_text=test_query,
    scoring_weights=scoring_weights,
)

print("\n=== Final Output ===")
print(result)


=== Running Abdominal Ultrasound (Gallbladder) Validation Test ===

[EventBus Error] Handler 'on_task_started' failed for event 'TaskStartedEvent': 'NoneType' object has no attribute 'key'

=== Final Output ===
=== Radiology Validation Report ===
Region: abdomen
ACR Used: No
Rubric Used: No
Weights: ACR=0.6, Rubric=0.4
Overall Adherence Score: Not Applicable
ACR Score: N/A
Rubric Score: N/A

Documentation Quality Assessment:
- No reference standards available for validation
- Report contains anatomical findings for gallbladder, biliary system, solid organs, and kidneys
- Measurements provided for kidneys and common hepatic duct

Areas for Improvement:
- Cannot assess without reference standards
- Validation requires either ACR guidelines or institutional rubric to be available

Notes:
- This report cannot be scored without access to validation standards
- To enable quality assessment, please provide either ACR guidelines or institutional rubric for the abdomen region
